# P05 schedule

## Preamble

This section contains package imports and general settings for the notebook.

In [1]:
import pandas as pd
import datetime as dt
import calendar
# import dateutil as du
import numpy as np
from pathlib import Path
import os
import re
from fuzzywuzzy import fuzz

# replace n with the number of columns/rows you want to see completely
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 400)

## User input
- Year 
- Path to the dirctory which contains the following documents:
    - Petra 3 schedule provided by Oliver Seeck
    - DOOR scheduling support - available after the ranking was confirmed in DOOR

In [2]:
year = 2023

# directory which contains all necessary files to import,
# starting from "./desycloud"
dc_path = Path('desycloud/documents/' + str(year) + '/p05_schedule/')
# path to Petra 3 schedule
dc_p3sch_path = dc_path / 'Schedule2023_draft_Status2022.08.18_shifted_timing.xlsx'
# path to DOOR scheduling support
dc_proposals_path_run1 = dc_path / '2023_1/door.scheduling.support.xlsx'
dc_proposals_path_run2 = dc_path / '2023_2/door.scheduling.support.xlsx'

# School holidays in Hamburg, Niedersachsen and Schleswig-Holstein
school_holidays = {"Hamburg":
                   {"Weihnachtferien": pd.date_range(start="2023-01-01", end="2023-01-06"),
                    "Winterferien": pd.date_range(start="2023-01-27", end="2023-01-27"),
                    "Osterferien": pd.date_range(start="2023-03-06", end="2023-03-17"),
                    "Pfingstferien": pd.date_range(start="2023-05-15", end="2023-05-19"),
                    "Sommerferien": pd.date_range(start="2023-07-13", end="2023-08-23"),
                    "Herbstferien": pd.date_range(start="2023-10-02", end="2023-10-27")},
                   "Niedersachsen":
                   {"Weihnachtferien": pd.date_range(start="2023-01-01", end="2023-01-06"),
                    "Winterferien": pd.date_range(start="2023-01-30", end="2023-01-31"),
                    "Osterferien": pd.date_range(start="2023-03-27", end="2023-04-11"),
                    "Pfingstferien1": pd.date_range(start="2023-05-19", end="2023-05-19"),
                    "Pfingstferien2": pd.date_range(start="2023-05-30", end="2023-05-30"),
                    "Sommerferien": pd.date_range(start="2023-07-06", end="2023-08-16"),
                    "Herbstferien1": pd.date_range(start="2023-10-02", end="2023-10-02"),
                    "Herbstferien2": pd.date_range(start="2023-10-16", end="2023-10-30")},
                   "Schleswig-Holstein":
                   {"Weihnachtferien":  pd.date_range(start="2023-01-01", end="2023-01-07"),
                    "Osterferien": pd.date_range(start="2023-04-06", end="2023-04-22"),
                    "Pfingstferien": pd.date_range(start="2023-05-19", end="2023-05-20"),
                    "Sommerferien": pd.date_range(start="2023-07-17", end="2023-08-26"),
                    "Herbstferien": pd.date_range(start="2023-10-16", end="2023-10-27")}
                   }

# Holidays in Hamburg
hh_holidays = {"Neujahr":  dt.date(2023, 1, 1),
               "Karfreitag": dt.date(2023, 4, 7),
               "Ostermontag": dt.date(2023, 4, 10),
               "Tag der Arbeit": dt.date(2023, 5, 1),
               "Christi Himmelfahrt": dt.date(2023, 5, 18),
               "Pingstmontag": dt.date(2023, 5, 29),
               "Tag der Deutschen Einheit": dt.date(2023, 10, 3),
               "Reformationstag": dt.date(2023, 10, 31),
               "1. Weihnachtsfeiertag": dt.date(2023, 12, 25),
               "2. Weihnachtsfeiertag": dt.date(2023, 12, 26)
               }

# Absences
absences = {"FW": [[dt.date(2023, 1, 2), dt.date(2023, 1, 8)],  # Weihnachtsferien
                   [dt.date(2023, 1, 28), dt.date(2023, 1, 31)],  # Winterferien N
                   [dt.date(2023, 1, 17), dt.date(2023, 1, 17)],  # Olivia Geburtstag
                   [dt.date(2023, 3, 15), dt.date(2023, 3, 15)],  # Piet Geburtstag
                   [dt.date(2023, 3, 30), dt.date(2023, 4, 11)],  # Osterferien
                   # [dt.date(2023, 4, 29), dt.date(2023, 5, 1)],  # 1. Mai (?) => JH
                   # [dt.date(2023, 5, 18), dt.date(2023, 5, 21)],  # Himmelfahrt (?) => JH
                   # [dt.date(2023, 5, 27), dt.date(2023, 5, 30)],  # Pfingstferien (?) => JH
                   [dt.date(2023, 7, 22), dt.date(2023, 8, 6)],  # Sommerferien
                   # [dt.date(2023, 9, 30), dt.date(2023, 10, 3)],  # Tag der deutsche Einheit (?) => JH
                   [dt.date(2023, 10, 23), dt.date(2023, 10, 31)],  # Herbstferien
                   [dt.date(2023, 12, 22), dt.date(2024, 1, 7)],  # Weihnachstferien
                  ],
            "JH": [
                   # [dt.date(2023, 4, 29), dt.date(2023, 5, 1)],  # 1. Mai (?) => JH
                   # [dt.date(2023, 5, 18), dt.date(2023, 5, 21)],  # Himmelfahrt (?) => JH
                   # [dt.date(2023, 5, 27), dt.date(2023, 5, 30)],  # Pfingstferien (?) => JH
                   # [dt.date(2023, 9, 30), dt.date(2023, 10, 3)],  # Tag der deutsche Einheit (?) => JH
                  ],
           }

In [3]:
# Choose the path depending if I'm at home or at work
system = os.name
if os.name == "nt":
    path = "D:/"
else:
    path = "/home/fwilde/"

p3sch_path = path / dc_p3sch_path
proposals_path_run1 = path / dc_proposals_path_run1
proposals_path_run2 = path / dc_proposals_path_run2

## Code

### Import Petra 3 schedule code

In [93]:
##################################################
# Class to work with the p05 schedule
##################################################

class p05sch:
    """
    P05 schedule class.

    Args:
        year <int>: Year for which the p05 schedule should be created
        p3sch_path <str>: path to the PETRA III schedule
        school_holidays <optional, dict>: dictionary containing the school holidays {Bundesland: {Ferienname: daterange}, ...}
        hh_holdays <optional, dict>: dictionary containing the Hamburg holdiays {Feiertagname: date, ...}
    """

    def __init__(self, year, p3sch_path, school_holidays=None, hh_holidays=None, absences=None, proposals_path=None):
        self.year = year
        self.schedule = None
        self.p3_modes = None
        self.p3_addinfo = None
        self.p3_stats = list()
        self.p3sch = pd.read_excel(p3sch_path)
        self.filter()
        self.school_holidays = school_holidays
        self.hh_holidays = hh_holidays
        self.absences = absences

        if not self.schedule:
            self.init()
        if school_holidays:
            self.add_school_holiday()
        if absences:
            self.add_absences()

    def filter(self):
        '''
        Strip imported Petra 3 schedule of everything except the Petra III modes and additional info.

        Args:
            p3sch <pandas.DataFrame>:  Petra III schedule read in by read_excel()

        Return:
            p3_modes <pandas.DataFrame>:  table with PETRA III modes
            p3_addinfo <pandas.DataFrame>:  table with additional info
        '''
        counter = 0
        machine_col = []
        addinfo_col = []
        months = ["December", "January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December.1"]
        for col in self.p3sch.iloc[[0]]:
            if col in months:
                machine_col.append(counter + 3)  # 3rd column after the month name column should contain the Petra III modes
                addinfo_col.append(counter + 4)  # 4th column after the month name column should contain the Petra III additional info
            counter += 1
        self.p3_modes = self.p3sch[self.p3sch.columns[machine_col]][1:32]  # fetch mode info from Petra III schedule. Everything beyond row 32 should not belong to the schedule.
        self.p3_addinfo = self.p3sch[self.p3sch.columns[addinfo_col]][1:32]  # fetch additional info from Petra III schedule. Everything beyond row 32 should not belong to the schedule.

        self.p3_modes.columns = months  # set column names to months
        self.p3_addinfo.columns = months  # set column names to months

        self.p3_modes.drop(["December"], axis=1, inplace=True)  # get rid of last years December column
        self.p3_addinfo.drop(["December"], axis=1, inplace=True)  # get rid of last years December column
        self.p3_modes.rename(columns={"December.1": "December"}, inplace=True)  # rename imported December.1 of current year to December
        self.p3_addinfo.rename(columns={"December.1": "December"}, inplace=True)  # rename imported December.1 of current year to December

        stats_col = self.p3sch.apply(lambda row: row.astype(str).str.contains('per run').any(), axis=1)
        stats_row_index = list(stats_col[stats_col == True].index)
        for entry in self.p3sch.iloc[stats_row_index[0]]:
            if type(entry) == int:
                self.p3_stats.append(entry)

    def inject_data(self, startdate, enddate, column, data):
        '''
        Injects data into a column of the schedule. Use data=np.nan to remove data from a data frame.

        Args:
            schedule <pandas.DataFrame>
            startdate <datetime.date>
            enddate <datetime.date>
            column <string>
            data <arbitrary>

        Return:
            True
        '''
        dr = pd.date_range(startdate, enddate)
        for date in dr:
            self.schedule.loc[date, column] = data

    def inject_list(self, startdate, enddate, column, data):
        '''
        Injects list data into a column of the schedule. If the list is shorter than the time range,
        missing values will be set to NaN.

        Args:
            startdate <datetime.date>
            enddate <datetime.date>
            column <string>
            data <list> <np.array> <pd.Series>

        Return:
            True
        '''
        dr = pd.date_range(startdate, enddate)  # generate a pandas series wiht dates
        lendiff = len(dr) - len(data)
        if lendiff > 0:  # if the data list is shorter than the list of dates, fill with the missing values with None
            try:
                data.append(None*lendiff)
            except:
                data = np.append(data, [None]*lendiff)

        if type(data) == list:  # in case data of Setup <list>, <np.array>
            for i, date in enumerate(dr):
                self.schedule.loc[date, column] = data[i]
        if type(data) == pd.Series:
            for i, date in enumerate(dr):  # in case data of Setup <pd.Series>
                self.schedule.loc[date, column] = data.iloc[i]

        self.schedule.replace(to_replace=[None], value=np.nan, inplace=True)

    def init(self):
        '''
        Generates an empty p05 schedule which only includes the Petra 3 modes / additional info.

        Args:
            year <int>

        Return:
            schedule <pandas.DataFrame>
        '''
        startdate = dt.date(self.year, 1, 1)
        enddate = dt.date(self.year, 12, 31)
        months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
        schedule_columns = ['School holidays', 'P3 modes', 'P3 info', 'Setup', 'Proposal', 'Application', 'Leader', 'PI', 'Local contact', 'Absence', 'Extra hours', 'Title', 'Conferences', 'Comment']
        self.schedule = pd.DataFrame(index=pd.date_range(startdate, enddate), columns=schedule_columns)
        for month in np.arange(12):
            daysinmonth = calendar.monthrange(year, month+1)[1]  # get number of days in current month
            month_modes = self.p3_modes[months[month]].iloc[:daysinmonth]
            month_addinfo = self.p3_addinfo[months[month]].iloc[:daysinmonth]
            self.inject_list(dt.date(self.year, month+1, 1), dt.date(self.year, month+1, daysinmonth), "P3 modes", month_modes)
            self.inject_list(dt.date(self.year, month+1, 1), dt.date(self.year, month+1, daysinmonth), "P3 info", month_addinfo)
        if self.school_holidays:
            self.add_school_holiday()
        if self.absences:
            self.add_absences()

    def add_school_holiday(self):
        '''
        Add school holidays to schedule.  Needs a self.school_holday dictionary to work.

        Args:
            None
        '''
        startdate = dt.date(self.year, 1, 1)
        enddate = dt.date(self.year, 12, 31)
        sh_col = pd.DataFrame(index=pd.date_range(startdate, enddate), columns=["School holidays"])  # grab school holidays column
        for day, holiday in sh_col.iterrows():  # 1. iterate over all days in the year, to compare each day with the school holidays
            holidays_in = ""  # this is the string with the bundesland holidays on a specific days (like "HNS")
            for bundesland, b_school_holidays in self.school_holidays.items():  # 2. iterate over bundesländer
                for holidayname, daterange in b_school_holidays.items():  # within each bundesland iterate over one specific holiday
                    if day in daterange:
                        holidays_in += bundesland[0]
            holidays_string_dict = {"": "", "HNS": "HNS", "H": 'H  ', "N": " N ", "S": "  S", "HN": "HN ", "HS": "H S", "NS": " NS"}
            self.schedule.at[day, "School holidays"] = holidays_string_dict[holidays_in]

    def add_absences(self):
        '''
        Add absences to schedule.  Needs an absences dictionary to work.

        Args:
            None
        '''
        startdate = dt.date(self.year, 1, 1)
        enddate = dt.date(self.year, 12, 31)
        absence_col = pd.DataFrame(index=pd.date_range(startdate, enddate), columns=["Absence"])  # grab Absence column
        for day, absence in absence_col.iterrows():  # 1. iterate over all days in the year, to compare each day with days of absence
            absence_string = ""  # this is the string with the absences on a specific days (like "JH FW")
            for person, absence_list in self.absences.items():  # 2. iterate over absences
                for date_range in absence_list:
                    dr = pd.date_range(start=date_range[0], end=date_range[1])
                    if day in dr:
                        absence_string += person + " "
            self.schedule.at[day, "Absence"] = absence_string

    def eval_proposals(self, proposals_instance, clear=True):
        '''
        Use a proposal class instance to add proposals to the schedule. A proposal is added
        when a start date for the proposal exists,

        Args:
            clear <boolean>: if true, the entire schedule is cleared before it is filled proposals
        '''
        if clear:
            self.init()  # Clear entire table
        for appid in proposals_instance.table.index:
            if not pd.isnull(proposals_instance.table["Start date"][appid]):
                days = int(proposals_instance.table["Shifts final"][appid] / 3)
                start_date = proposals_instance.table["Start date"][appid]
                dayspan, i = 0, 0
                while i < days:  # if service days or the like are in the date range, extend the date range
                    currdate = start_date+dt.timedelta(days=dayspan)
                    dayspan += 1
                    if self.schedule.loc[currdate, "P3 modes"] in ["multi", 40]:
                        i += 1

                for day in range(dayspan):
                    currdate = start_date+dt.timedelta(days=day)
                    currapp = self.schedule.loc[currdate, "Application"]
                    self._check_absences(appid, currdate, proposals_instance.table.loc[appid, "Local contact"])
                    if not pd.isna(currapp):  # Check if there is already an application on this date
                        print("WARNING: {}: Collision for Applications {} and {}".format(currdate, int(appid), int(currapp)))
                    if self.schedule.loc[currdate, "P3 modes"] in ["multi", 40]:
                        # self._check_absences(appid, currdate, proposals_instance.table.loc[appid, "Local contact"])
                        for col in ["Setup", "Proposal", "Application", "Leader", "PI", "Local contact", "Title", "Comment"]:
                            if col == "Application":
                                self.schedule.loc[currdate, col] = appid
                            else:
                                self.schedule.loc[currdate, col] = proposals_instance.table.loc[appid, col]

    def _check_absences(self, appid, date, local_contact):
        '''
        Checks whether a beamtime lies in the absences dated of the assigned local contact.

        Args:
            appid <int>: Application ID
            date <dt.datetime>: Date which is to be checked
            local_contact <str>: Acronym of the local contact ("FW", "JH", etc.)
        '''
        if local_contact:
            for absence_pair in self.absences[local_contact]:
                absence_range = pd.date_range(start=absence_pair[0], end=absence_pair[1])
                if date in absence_range:
                    print("WARNING: {}: Local contact {} is absent for scheduled application {}.".format(date, local_contact, appid))

    def _color_index(self, date):
        if date.weekday() == 5:
            color = 'blue'
        elif date.weekday() == 6:
            color = 'mediumblue'
        elif date.date() in list(hh_holidays.values()):
            color = 'orange'
        else:
            color = None

        return 'background-color: {}'.format(color)

    def _color_holidays(self, date):
        if date.date() in list(hh_holidays.values()):
            color = 'orange'
        else:
            color = None
        return 'background-color: %s' % color

    def _color_p3modes(self, mode):
        if mode in ['multi', 40]:
            color = 'seagreen'
        elif mode in ['tr']:
            color = 'green'
        else:
            color = 'maroon'
        return 'background-color: %s' % color


    def _style_schedule(self, styler):
        '''
        Schedule style, used in self.show()
        '''
        styles = [dict(selector="caption", props=[("font-size", "150%"),
                                          ("font-weight", "bold")])]
        styler.set_caption('P05 schedule {}'.format(year)).set_table_styles(styles)
        styler.format(na_rep='')  # don't print NaNs
        styler.applymap_index(self._color_index)
        styler.applymap(self._color_p3modes, subset='P3 modes')
        styler.format(precision=0, subset='Application', na_rep='')
        styler.format('{:3s}', subset='School holidays', na_rep='')
        styler.set_sticky(axis="columns")
        styler.format_index("{:%Y-%m-%d}")  # Remove the time from the date which is introduced by styler


        return styler

    def show(self, startdate=None, enddate=None):
        '''
        Display the schedule table.

        Args:
            startdate <dt.datetime>: Optional, default None. First date in the the displayed schedule
            enddate <dt.datetime>: Optional, default None. Last date in the displayed schedule
        '''
        # self.schedule.loc[dt.datetime(2023, 2, 10):dt.datetime(2023, 8, 1)].style.pipe(colorize_schedule)
        color_schedule = self.schedule.loc[startdate:enddate].style.pipe(self._style_schedule)
        display(color_schedule)

### Import proposal table code

In [434]:
##################################################
# Functions to work with table of proposals
##################################################
class proposals:
    '''
    The proposals class can read, filter and modify a table with all P05 proposals.

    Args:
        proposals_path <str>: Path to the scheduling support table (list of proposals).
        p3sch_path <str>: path to the PETRA III schedule
    '''

    def __init__(self, proposals_path, p3sch_path, year=None, run=1):
        self.unfiltered_table = pd.read_excel(proposals_path)
        self.p3sch = pd.read_excel(p3sch_path)
        self.p3_modes = None
        self.p3_stats = list()
        self.beamtime_distribution = pd.DataFrame(columns=['days', 'shifts', 'fraction'])
        self.table = None
        self.year = year
        self.run = run

    def filter(self, filter_string='auto', include=[], p3_stats=None):
        '''
        Strip scheduling support table (list of proposal) from all but the current proposals.

        Args:
            filter_string <str>: String used to filter scheduling support table in the "Ranking" column.
                                 If set to "auto", the table is filtered based on the
                                 highest sum of the call and year integers in the
                                 "Ranking" column (eg. "2nd call 2022" -> 2024).
            include <list>: Add a list of Application IDs, that should also be included in the filtered_proposal_table
        '''
        if filter_string == "auto":
            mask_petra = self.unfiltered_table["Ranking"].str.contains("PETRA")  # create mask based on the string "PETRA"
            mask_petra.fillna(False, inplace=True)  # Replace NaN with False in mask
            filtered_proposals = self.unfiltered_table[mask_petra]  # filter proposals based on mask_petra
            r_filtered_proposals = filtered_proposals.reset_index(drop=True)  # rebuild index from 0.. in filtered_proposals
            numbers_in_ranking = filtered_proposals["Ranking"].str.findall(r'([0-9]+)')  # new dataframe generating a list with all numbers (as strings) in the "Ranking" column
            split_numbers_in_ranking = pd.DataFrame(numbers_in_ranking.to_list(), columns=['ranking', 'call', 'year'])  # new dataframe based o
            split_numbers_in_ranking["sum_call_year"] = split_numbers_in_ranking["call"].astype("int") + split_numbers_in_ranking["year"].astype("int")
            max_nir = max(split_numbers_in_ranking["sum_call_year"])
            split_numbers_in_ranking["mask"] = (split_numbers_in_ranking["sum_call_year"] == max_nir)
            mask_newest = split_numbers_in_ranking["mask"]
            self.table = r_filtered_proposals[mask_newest]
        else:
            mask_petra = self.unfiltered_table["Ranking"].str.contains(filter_string)  # create mask based on the filter_string
            mask_petra.fillna(False, inplace=True)  # Replace NaN with False in mask
            filtered_proposals = self.unfiltered_table[mask_petra]  # filter proposals based on mask_petra
            self.table = filtered_proposals.reset_index(drop=True)  # rebuild index from 0.. in filtered_proposals
        include_proposals = self.unfiltered_table[self.unfiltered_table["Application"].isin(include)].copy()
        include_proposals.loc[:, "Ranking (numbers)"] = -1
        # Add column sthat will be needed later on
        self.table = pd.concat([self.table, include_proposals])
        self.table.set_index("Application", inplace=True)  # prevent multiple identical Application IDs
        self.table.insert(8, "Shifts final", None)
        for appid, shifts_assigned in self.table["Shifts assigned"].items():
            if shifts_assigned == 72:
                shifts_assigned = 9
            self.table.at[appid, "Shifts final"] = shifts_assigned
        self.table.insert(9, "Start date", pd.NaT)
        self.table.insert(19, "Unacceptable list", pd.NaT)
        self.table.insert(10, "Local contact", None)
        self.table["Comment"] = None

        stats_col = self.p3sch.apply(lambda row: row.astype(str).str.contains('per run').any(), axis=1)
        stats_row_index = list(stats_col[stats_col == True].index)
        for entry in self.p3sch.iloc[stats_row_index[0]]:
            if type(entry) == int:
                self.p3_stats.append(entry)
        self.beamtime_distribution.loc["user run", "days"] = self.p3_stats[0]

        # fetch PETRA 3 schedule modes
        counter = 0
        machine_col = []
        addinfo_col = []
        months = ["December", "January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December.1"]
        for col in self.p3sch.iloc[[0]]:
            if col in months:
                machine_col.append(counter + 3)  # 3rd column after the month name column should contain the Petra III modes
            counter += 1
        p3_modes_raw = self.p3sch[self.p3sch.columns[machine_col]][1:32]  # fetch mode info from Petra III schedule. Everything beyond row 32 should not belong to the schedule.
        p3_modes_raw.columns = months  # set column names to months
        p3_modes_raw.drop(["December"], axis=1, inplace=True)  # get rid of last years December column
        p3_modes_raw.rename(columns={"December.1": "December"}, inplace=True)  # rename imported December.1 of current year to December
        startdate = dt.date(self.year, 1, 1)
        enddate = dt.date(self.year, 12, 31)
        months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
        schedule_columns = ['P3 modes']
        self.p3_modes = pd.DataFrame(index=pd.date_range(startdate, enddate), columns=schedule_columns)
        for month in np.arange(12):
            daysinmonth = calendar.monthrange(year, month+1)[1]  # get number of days in current month
            month_modes = p3_modes_raw[months[month]].iloc[:daysinmonth]
            dr = pd.date_range(dt.datetime(self.year, month+1, 1), dt.datetime(self.year, month+1, daysinmonth))
            for i, date in enumerate(dr):  # in case data of Setup <pd.Series>
                self.p3_modes.loc[date, "P3 modes"] = month_modes.iloc[i]

    def modify_beamtime_distribution(self, nano_days=None, micro_days=None, nano_fraction=0.5, micro_fraction=None, nano_user=0.8, micro_user=0.8, nano_comm_inh=0.2, micro_comm_inh=0.2, nano_industry_days=1, micro_industry_days=2):
        user_run_days = self.beamtime_distribution.loc["user run", "days"]
        if nano_days:
            micro_days = int(user_run_days - nano_days)
        elif micro_days:
            nano_days = int(micro_days - nano_days)
        elif nano_fraction:
            nano_days = int(np.round(nano_fraction * user_run_days))
            micro_days = int(user_run_days - nano_days)
        elif micro_fraction:
            micro_days = int(np.round(micro_fraction * user_run_days))
            nano_days = int(user_run_days - micro_days)

        self.beamtime_distribution.loc["nano total", "days"] = nano_days
        self.beamtime_distribution.loc["micro total", "days"] = micro_days

        if nano_user < 1:
            nano_user_days = int(np.ceil(nano_days * nano_user) - nano_industry_days)
            nano_cominh_days = int(nano_days - nano_user_days - nano_industry_days)
        else:
            nano_user_days = int(nano_user - nano_industry_days)
            nano_cominh_days = int(nano_days - nano_user_days - nano_industry_days)

        if micro_user < 1:
            micro_user_days = int(np.ceil(micro_days * micro_user) - micro_industry_days)
            micro_cominh_days = int(micro_days - micro_user_days - micro_industry_days)
        else:
            micro_user_days = int(micro_user - micro_industry_days)
            micro_cominh_days = int(micro_days - micro_user_days - micro_industry_days)

        self.beamtime_distribution.loc["nano user", "days"] = nano_user_days
        self.beamtime_distribution.loc["micro user", "days"] = micro_user_days
        self.beamtime_distribution.loc["nano comm inh", "days"] = nano_cominh_days
        self.beamtime_distribution.loc["micro comm inh", "days"] = micro_cominh_days
        self.beamtime_distribution.loc["nano industry", "days"] = nano_industry_days
        self.beamtime_distribution.loc["micro industry", "days"] = micro_industry_days
        self.beamtime_distribution.loc["user run", "fraction"] = np.round(user_run_days/user_run_days * 100, decimals=2)
        self.beamtime_distribution.loc["nano total", "fraction"] = np.round(nano_days/user_run_days * 100, decimals=2)
        self.beamtime_distribution.loc["micro total", "fraction"] = np.round(micro_days/user_run_days * 100, decimals=2)
        self.beamtime_distribution.loc["nano user", "fraction"] = np.round(nano_user_days/nano_days * 100, decimals=2)
        self.beamtime_distribution.loc["micro user", "fraction"] = np.round(micro_user_days/micro_days * 100, decimals=2)
        self.beamtime_distribution.loc["nano comm inh", "fraction"] = np.round(nano_cominh_days/nano_days * 100, decimals=2)
        self.beamtime_distribution.loc["micro comm inh", "fraction"] = np.round(micro_cominh_days/micro_days * 100, decimals=2)
        self.beamtime_distribution.loc["nano industry", "fraction"] = np.round(nano_industry_days/nano_days * 100, decimals=2)
        self.beamtime_distribution.loc["micro industry", "fraction"] = np.round(micro_industry_days/micro_days * 100, decimals=2)

        self.beamtime_distribution["shifts"] = 3*self.beamtime_distribution["days"]

        check_total_days = (nano_days + micro_days == user_run_days)
        check_nano_days = (nano_user_days + nano_cominh_days + nano_industry_days == nano_days)
        check_micro_days = (micro_user_days + micro_cominh_days + micro_industry_days == micro_days)
        if not (check_total_days and check_nano_days and check_micro_days):
            print("Something does not add up:")
            if not check_total_days:
                print("total days: {} + {} != {}".format(micro_days, nano_days, user_run_days))
            if not check_micro_days:
                print("micro days: {} + {} + {} != {}".format(micro_user_days, micro_cominh_days, micro_industry_days, micro_days))
            if not check_nano_days:
                print("nano days: {} + {} + {} !- {}".format(nano_user_days, nano_cominh_days, nano_industry_days, nano_days))

    def convert_unacceptables(self, verbose=False):
        '''
        Converts user provided unacceptable dates into a date range, which can be checked against beamtime dates. The result is
        a date range (list of dates) in the "UNacceptable list" column of the proposal table.
        Dates are converted automatically, if they have the fowolling format:
        - Dates or date ranges are separated by either "," or ";"
        - Entire months can be specified as string, like "march" or "march; april", capital letters are ignored
        - Date ranges are assigned with "-" (like "1.4-20.4")
        - Dates may have the format "1.4", "1.4.", "1.4.2023"
        - In date ranges the starting date may be only "1." as in "1-10.4"

        Args:
            verbose <boolean>: Optional, default False. Prints out information about the conversion
        '''
        months = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]
        for appid, unacceptable in self.table["Unacceptable"].items():
            if verbose:
                print("Leader/ PI / beamtime: {} / {} / {}".format(self.table.loc[appid, "Leader"], self.table.loc[appid, "PI"], appid))
                print("user input: \"{}\"".format(unacceptable))
            if pd.isnull(unacceptable):
                pass
            else:
                split_separator = re.split(r'[,;]+', unacceptable)
                split_range_symbol = None
                unacceptable_list = []
                # Try first to extract single month names and generate a daterange from that
                for dt_range in split_separator:
                    fz_result = [fuzz.partial_ratio(item, dt_range) for item in months]
                    try:
                        match = fz_result.index(100) + 1
                        if match:
                            daysinmonth = calendar.monthrange(year, match)[1]  # get number of days in current month
                            startdate = "{}-{}-{}".format(year, match, 1)
                            enddate = "{}-{}-{}".format(year, match, daysinmonth)
                            unacceptable_list.append(pd.date_range(start=startdate, end=enddate))
                            continue
                    except:
                        # then go ahead and try to convert dates directly
                        split_separator = dt_range.replace(" to ", "-")
                        split_range_symbol = re.split(r'[-]+', split_separator)
                        contains_dot = ["." in item for item in split_range_symbol]
                        if True in contains_dot:
                            startdate = [item for item in split_range_symbol[0].split(".") if item]  # split date by . and make sure that there are no empty strings
                            enddate = [item for item in split_range_symbol[1].split(".") if item]  # split date by . and make sure that there are no empty strings
                            startday = startdate[0].replace(" ", "")
                            endday = enddate[0].replace(" ", "")
                            if len(enddate) > 1:
                                endmonth = enddate[1]
                            if len(startdate) > 1:
                                startmonth = startdate[1].replace(" ", "")
                            else:
                                startmonth = endmonth
                            unacceptable_list.append(pd.date_range(start="{}-{}-{}".format(year, startmonth, startday), end="{}-{}-{}".format(year, endmonth, endday)))
                            self.table.at[appid, "Unacceptable list"] = unacceptable_list
                        else:  # give up if that fails
                            print("WARNING: Application {} ({} / {}): Couldn't convert \"{}\" to datetime.".format(appid, self.table.loc[appid, "Leader"], self.table.loc[appid, "PI"], unacceptable))

                if unacceptable_list:  # flatten unacceptable list
                    unacceptable_list = pd.DatetimeIndex([item for sublist in unacceptable_list for item in sublist])
                    self.table.at[appid, "Unacceptable list"] = unacceptable_list
                    if verbose:
                        print("List of unaccepatable dates: \n{}".format(unacceptable_list))
            if verbose:
                print("=====================================================")

    def modify_unacceptables(self, appid, entry):
        '''
        Modifies a user provided unacceptable date. This should be used if the auto detection for the unaccetable dates fails.

        Args:
            appid <int>: Application ID
            entry <str>: The new unacceptable entry as string (format eg.: "march, april", "1.4.-10.4.")
        '''
        self.table.at[appid, "Unacceptable"] = entry

    def show(self, exclude=['Collaboration', 'Ranking', 'Beamline', 'Submitted', 'Filling mode (bunches)', 'Beam size', 'Unacceptable list'], include=[], split=True):
        '''
        Show the filtered proposal table and optionally exclude columns. If an include list is provided only
        the columns in the inlude list will be displayed.

        Args:
            exclude <str list>: Optional. List of columns that should be excluded in display. Defaults to
                                ['Collaboration','Ranking', 'Beamline','Submitted','Filling mode (bunches)','Beam size']
            include <str list>: Optional. List of columns that shoud be included in display.  Defauilts to [].
        '''
        if split:
            micro = self.table[self.table['Setup'].str.contains('Microtomography')]
            nano = self.table[self.table['Setup'].str.contains('Nanotomography')]
            setup_dict = {"Microtomography":micro, "Nanotomography":nano}
            for setup_name, setup_table in setup_dict.items():
                if not include:
                    style_proposals = setup_table.loc[:, ~self.table.columns.isin(exclude)].sort_values(by=["Setup", "Ranking (numbers)", "Proposal"]).style.pipe(self._style_proposals, setup=setup_name)
                else:
                    style_proposals = setup_table.loc[:, self.table.columns.isin(include)].style.pipe(self._style_proposals, setup=setup_name)
                display(style_proposals)

        else:
            if not include:
                style_proposals = self.table.loc[:, ~self.table.columns.isin(exclude)].sort_values(by=["Ranking (numbers)", "Proposal"]).style.pipe(self._style_proposals, setup=None)
            else:
                style_proposals = self.table.loc[:, self.table.columns.isin(include)].style.pipe(self._style_proposals, setup=None)
            display(style_proposals)

    def assign(self, appid, shifts=9, local_contact=None, start_date=None, comment=None):
        '''
        Assign a local contact, number of shifts, Start date and a comment to an application ID and enter these values in the proposal table.

        Args:
            appid <int>: Application ID
            number_of_shifts <optional, int>: How many shifts should be assigned to a beamtime, defaults to 9 (!)
            local_contact <optional, str>: Local contact (initials)
            start_date <optional, dt.date>: When shoudl the beamtime begin
            comment <optional, str>: A comment tha should go along with the beamtime
        '''
        if shifts:
            self.table.at[appid, "Shifts final"] = shifts
        else:
            self.table.at[appid, "Shifts final"] = self.table.loc[appid, "Shifts assigned"]
        if local_contact:
            self.table.at[appid, "Local contact"] = local_contact
        if start_date:
            self.table.at[appid, "Start date"] = start_date
            # beamtime_dates = [(start_date+dt.timedelta(days=i)).strftime("%Y-%m-%d") for i in range(int(shifts/3))]
            if self._check_unacceptables(appid):
                print("Warning: Beamtime {} ({} / {}): startdate {} lies with the unacceptable dates.".format(appid, self.table.loc[appid, "Leader"], self.table.loc[appid, "PI"], start_date.strftime("%Y-%m-%d")))
        if comment:
            self.table.at[appid, "Comment"] = comment

    def add_beamtime(self, appid, proposal=None, setup=None, title=None, leader=None, pi=None, ranking=100, shifts=None, start_date=None, comment=None):
        '''
        Add a new beamtime (eg. commissioning or inhouse).

        Args:
            appid <int>: Application ID
            proposal <str>: Optional, default None, Proposal ID
            setup <str>: Optional, default None. One of "micro" or "nano". Will be replaced with long setup string (eg. "Microtomography (EH2)")
            title <str>: Optional, default None. Title of the proposal.
            leader <str>: Optional, default None. Proposal leader.
            pi <str>: Optional, default None. Principal investigator.
            ranking <int>: Optional, default 100. Ranking of the proposal.
            shifts <int>: Optional, default None. Granted shifts for a proposal.
            start_date <dt.datetime>: Optionalm default None. Start date of the proposal
            comment <str>: Optional, default None. Comment.
        '''
        setup_dict = {'micro': 'Microtomography (EH2)', 'nano': 'Nanotomography (EH1)'}
        self.table.loc[appid] = {'Proposal': proposal, 'Setup': setup_dict[setup], 'Ranking (numbers)': ranking, 'Title': title, 'Leader': leader, 'PI': pi, 'Shifts assigned': shifts, 'Shifts final': shifts, 'Start date': start_date, "Comment": comment}

    def print_assign_commands(self, instance_name):
        '''
        Print commands to assign local contacts and dates to beamtimes with predefined values. The commands can
        than be copied and used to modify beamtime assignments.
        '''
        print_table = self.table[["Setup", "Ranking (numbers)", "Leader", "PI", "Shifts assigned"]].sort_values(by=["Setup", "Ranking (numbers)"])
        #display(print_table)
        for appid, setup in print_table["Setup"].items():
            print("#  {}.assign({}, local_contact=\"XX\", start_date=dt.datetime({}, 1, 1), shifts={})  # {} / {}, {}".format(instance_name, appid, self.year, print_table.loc[appid, "Shifts assigned"], print_table.loc[appid, "Leader"], print_table.loc[appid, "PI"], print_table.loc[appid, "Setup"]))

    def _check_unacceptables(self, appid):
        '''
        Checks if the beamtime dates collide with the user provided unacceptable dates

        Args:
            appid <int>: Application ID
        '''
        days = int(self.table["Shifts final"][appid] / 3)
        start_date = self.table["Start date"][appid]
        dayspan, i = 0, 0
        while i < days:  # if service days or the like are in the date range, extend the date range
            currdate = start_date+dt.timedelta(days=dayspan)
            dayspan += 1
            if self.p3_modes.loc[currdate, 'P3 modes'] in ["multi", 40]:
                i += 1

        for day in range(dayspan):
            currdate = start_date+dt.timedelta(days=day)
            if type(pd.isna(self.table.loc[appid, "Unacceptable list"])) is not bool:
                if currdate in self.table.loc[appid, 'Unacceptable list']:
                    print("WARNING: {}: Applications {} is in the unacceptable time range.".format(currdate, int(appid)))

    def _color_cutoff_micro(self, appid):
        micro = self.table[self.table['Setup'].str.contains('Microtomography')]
        micro = micro[(micro["Proposal"].str.contains("micro comm") == False)]
        if appid in micro.index.to_series():
            micro_user_sum = micro.sort_values(by=["Ranking (numbers)"]).loc[:appid, "Shifts final"].sum()
            if micro_user_sum > self.beamtime_distribution.loc["micro user", "shifts"]:
                color = 'maroon'
            else:
                color = 'seagreen'
        else:
            color = None
        return 'background-color: %s' % color

    def _color_cutoff_nano(self, appid):
        nano = self.table[self.table['Setup'].str.contains('Nanotomography')]
        nano = nano[(nano["Proposal"].str.contains("nano comm") == False)]
        if appid in nano.index.to_series():
            nano_user_sum = nano.sort_values(by=["Ranking (numbers)"]).loc[:appid, "Shifts final"].sum()
            if nano_user_sum > self.beamtime_distribution.loc["nano user", "shifts"]:
                color = 'maroon'
            else:
                color = 'seagreen'
        else:
            color = None
        return 'background-color: %s' % color
    
    def _color_cutoff(self, appid):
        nano_comm_filter = self.table[(self.table["Proposal"].str.contains("nano comm") == False)]
        comm_filter = nano_comm_filter[(nano_comm_filter["Proposal"].str.contains("micro comm") == False)]
        if appid in comm_filter.index.to_series():
            user_sum = comm_filter.sort_values(by=["Ranking (numbers)"]).loc[:appid, "Shifts final"].sum()
            if user_sum > self.beamtime_distribution.loc["nano user", "shifts"] + self.beamtime_distribution.loc["micro user", "shifts"]:
                color = 'maroon'
            else:
                color = 'seagreen'
        else:
            color = None
        return 'background-color: %s' % color

    def _style_proposals(self, styler, setup=None):
        '''
        Proposal table style, used in self.show()

        Args:
            setup <str>: Optional, either "Nanotomography" or "Mictotomography"
        '''
        styles = [dict(selector="caption", props=[("font-size", "150%"),
                                          ("font-weight", "bold")])]
        if setup:
            caption = setup + ' proposals {}, run {}'.format(self.year, self.run)
        else:
            caption = 'Proposals {}, run {}'.format(self.year, self.run)
        styler.set_caption(caption).set_table_styles(styles)
        if setup == "Microtomography":
            styler.applymap_index(self._color_cutoff_micro)
        if setup == "Nanotomography":
            styler.applymap_index(self._color_cutoff_nano)
        if not setup:
            styler.applymap_index(self._color_cutoff)
        styler.format(na_rep='', precision=0)  # don't print NaNs
        styler.set_sticky(axis="columns")

        return styler

# Initialize schedule and proposal_table

In [435]:
sch = p05sch(2023, p3sch_path, school_holidays=school_holidays, hh_holidays=hh_holidays, absences=absences)

In [450]:
# This part only works, if the scheduling support (list of proposals) is available
# The order of execution is important.

# proposals = filter_proposals(proposals, filter_string="2nd call 2022")
pro_run1 = proposals(proposals_path_run1, p3sch_path, year, run=1)

# Add applications that are also in the unfiltered proposal table
pro_run1.filter(include=[11016379, 11016663, 11016664, 11016665, 11016666, 11016667, 11016668], filter_string="2nd call 2022")

# define beamtime_distribution
pro_run1.modify_beamtime_distribution(nano_days=34, nano_user=0.7)

# Add commissioning, inhouse, industry etc
pro_run1.add_beamtime(101, proposal='micro comm', setup='micro', title='micro commissioning', shifts=6)
pro_run1.add_beamtime(102, proposal='micro comm', setup='micro', title='micro commissioning', shifts=6)
pro_run1.add_beamtime(103, proposal='micro comm', setup='micro', title='micro commissioning', shifts=3)
pro_run1.add_beamtime(104, proposal='micro comm', setup='micro', title='micro commissioning', shifts=3)

pro_run1.add_beamtime(201, proposal='nano comm', setup='nano', title='nano commissioning', shifts=6)
pro_run1.add_beamtime(11013127, proposal='I-20211523 EC', setup='micro', leader="Sket", pi="Sket", title='Grain boundary study in hexagonal microstructures by X-ray radiography and tomography during directional solidification', shifts=9, ranking=-1) 

# Modify unaceptables
pro_run1.modify_unacceptables(11016412, None)
pro_run1.modify_unacceptables(11016352, '01.02-15.03')
pro_run1.modify_unacceptables(11016060, 'january, february, march')
pro_run1.modify_unacceptables(11015941, 'january, february, march')
pro_run1.modify_unacceptables(11015918, '01.05-31.05')
pro_run1.modify_unacceptables(11015832, '09.01-20.01')
pro_run1.convert_unacceptables(verbose=False)

# This next command is only needed once to prepare the assignment of proposals
# Copy the output of this command below
# pro_run1.print_assign_commands(instance_name="pro_run1")

# Assign local contact, start date, shifts and comment to appilcations

# Microtomography
#################
#  pro_run1.assign(11016668, local_contact="JH", start_date=dt.datetime(2023, 1, 1), shifts=72)  # Wipfler / Hammel, Microtomography (EH2)
#  pro_run1.assign(11016667, local_contact="JH", start_date=dt.datetime(2023, 1, 1), shifts=72)  # Wipfler / Hammel, Microtomography (EH2)
pro_run1.assign(11016666, local_contact="JH", start_date=dt.datetime(2023, 2, 27), shifts=9)  # Herzen / Hammel, Microtomography (EH2)
pro_run1.assign(11016665, local_contact="JH", start_date=dt.datetime(2023, 2, 24), shifts=9)  # Herzen / Hammel, Microtomography (EH2)
#  pro_run1.assign(11016664, local_contact="JH", start_date=dt.datetime(2023, 1, 1), shifts=72)  # Wipfler / Hammel, Microtomography (EH2)
#  pro_run1.assign(11016663, local_contact="JH", start_date=dt.datetime(2023, 1, 1), shifts=72)  # Wipfler / Hammel, Microtomography (EH2)
#  pro_run1.assign(11016379, local_contact="FW", start_date=dt.datetime(2023, 1, 1), shifts=6)  # Rout / Rout, Microtomography (EH2)
pro_run1.assign(11013127, local_contact="FW", start_date=dt.datetime(2023, 7, 7), shifts=9)  # Sket / Sket, Microtomography (EH2)
#  pro_run1.assign(11016060, local_contact="JH", start_date=dt.datetime(2023, 1, 1), shifts=9)  # Hammel / Naumann, Microtomography (EH2)
#  pro_run1.assign(11016358, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=6)  # Blawert / Fazel, Microtomography (EH2)
#  pro_run1.assign(11016384, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=12)  # Wieland / Hindenlang, Microtomography (EH2)
#  pro_run1.assign(11015941, local_contact="FW", start_date=dt.datetime(2023, 1, 1), shifts=12)  # Manke / SUN, Microtomography (EH2)
#  pro_run1.assign(11016078, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=15)  # Adelung / Zeller-Plumhoff, Microtomography (EH2)
#  pro_run1.assign(11015864, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=9)  # Mayer / Mayer, Microtomography (EH2)
pro_run1.assign(11016439, local_contact="FW", start_date=dt.datetime(2023, 3, 3), shifts=9)  # Ziesche / Ziesche, Microtomography (EH2)
#  pro_run1.assign(11016279, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=6)  # Dultz / Dultz, Microtomography (EH2)
pro_run1.assign(11016144, local_contact="FW", start_date=dt.datetime(2023, 7, 14), shifts=9)  # Pistidda / Karimi, Microtomography (EH2)
#  pro_run1.assign(11015832, local_contact="FW", start_date=dt.datetime(2023, 1, 1), shifts=9)  # Manke / Osenberg, Microtomography (EH2)
#  pro_run1.assign(11016289, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=9)  # Engqvist / Engqvist, Microtomography (EH2)
#  pro_run1.assign(11016351, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=3)  # Hejnol / Hejnol, Microtomography (EH2)
#  pro_run1.assign(11015942, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=6)  # Blasco-Costa / Blasco-Costa, Microtomography (EH2)
#  pro_run1.assign(11015918, local_contact="FW", start_date=dt.datetime(2023, 1, 1), shifts=9)  # Manke / Paulisch, Microtomography (EH2)
#  pro_run1.assign(11016301, local_contact="FW", start_date=dt.datetime(2023, 1, 1), shifts=12)  # Manke / Arlt, Microtomography (EH2)
#  pro_run1.assign(11016415, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=9)  # Engelkes / Busse, Microtomography (EH2)
pro_run1.assign(101, local_contact="", start_date=dt.datetime(2023, 2, 22), shifts=6)  # nan / nan, Microtomography (EH2)
pro_run1.assign(102, local_contact="", start_date=dt.datetime(2023, 3, 6), shifts=6)  # nan / nan, Microtomography (EH2)
pro_run1.assign(103, local_contact="", start_date=dt.datetime(2023, 7, 6), shifts=3)  # nan / nan, Microtomography (EH2)
pro_run1.assign(104, local_contact="", start_date=dt.datetime(2023, 7, 13), shifts=3)  # nan / nan, Microtomography (EH2)

# Nanotomography
################
#  pro_run1.assign(11016412, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=12)  # Hesse / Hesse, Nanotomography (EH1)
#  pro_run1.assign(11016342, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=9)  # Zippo / Zippo, Nanotomography (EH1)
#  pro_run1.assign(11015833, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=9)  # Konchakova / Konchakova, Nanotomography (EH1)
#  pro_run1.assign(11016480, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=12)  # Greving / Herzen, Nanotomography (EH1)
#  pro_run1.assign(11016186, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=9)  # Tolnai / Tolnai, Nanotomography (EH1)
#  pro_run1.assign(11015933, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=12)  # Ni / Ni, Nanotomography (EH1)
#  pro_run1.assign(11016352, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=3)  # Oliveira / Oliveira, Nanotomography (EH1)
#  pro_run1.assign(11016103, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=9)  # Zeng / Xu, Nanotomography (EH1)
#  pro_run1.assign(11016148, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=9)  # Soederberg / Gordeyeva, Nanotomography (EH1)
pro_run1.assign(201, local_contact="", start_date=dt.datetime(2023, 3, 9), shifts=6)  # nan / nan, Microtomography (EH2)

display(pro_run1.beamtime_distribution.style.format(precision=1).set_caption("Beamtime distribution run 1").set_table_styles([dict(selector="caption", props=[("font-size", "150%"),("font-weight", "bold")])]))
pro_run1.show(split=True)

,days,shifts,fraction
user run,109,327,100.0
nano total,34,102,31.2
micro total,75,225,68.8
nano user,23,69,67.7
micro user,58,174,77.3
nano comm inh,10,30,29.4
micro comm inh,15,45,20.0
nano industry,1,3,2.9
micro industry,2,6,2.7


,Proposal,Setup,Ranking (numbers),Shifts applied,Shifts assigned,Shifts final,Start date,Local contact,Title,Leader,PI,Energy,Preferred dates,Unacceptable,Comment
Application,,,,,,,,,,,,,,,
11016664,BAG-20210019,Microtomography (EH2),-1,6,72,9,,,Understanding different scales oft the ecosystem ?amber forest? using SRμCT,Wipfler,Hammel,18 keV,,,
11016663,BAG-20210019,Microtomography (EH2),-1,12,72,9,,,Understanding different scales oft the ecosystem ?amber forest? using SRμCT,Wipfler,Hammel,18 keV,,,
11016666,BAG-20211054,Microtomography (EH2),-1,6,72,9,2023-02-27 00:00:00,JH,High-sensitivity quantitative phase-contrast tomography for biomedical and environmental applications,Herzen,Hammel,30 keV,,,
11016665,BAG-20211054,Microtomography (EH2),-1,12,72,9,2023-02-24 00:00:00,JH,High-sensitivity quantitative phase-contrast tomography for biomedical and environmental applications,Herzen,Hammel,30 keV,,,
11016668,BAG-20211055,Microtomography (EH2),-1,6,72,9,,,The arthropod tracheal system in the context of evolution and functional morphology in biology and medicine.,Wipfler,Hammel,30 keV,,,
11016667,BAG-20211055,Microtomography (EH2),-1,12,72,9,,,The arthropod tracheal system in the context of evolution and functional morphology in biology and medicine.,Wipfler,Hammel,25 keV,,,
11013127,I-20211523 EC,Microtomography (EH2),-1,,9,9,2023-07-07 00:00:00,FW,Grain boundary study in hexagonal microstructures by X-ray radiography and tomography during directional solidification,Sket,Sket,,,,
11016379,I-20221317,Microtomography (EH2),-1,6,6,6,,,Chromite grains in Antarctic micrometeorites: Constraining flux of extraterrestrial materials to Earth,Rout,Rout,18 keV,,,
11016060,I-20221037,Microtomography (EH2),1,9,9,9,,,In vivo imaging of the functional morphology of feeding in millipedes,Hammel,Naumann,30-33,May to July,"january, february, march",


,Proposal,Setup,Ranking (numbers),Shifts applied,Shifts assigned,Shifts final,Start date,Local contact,Title,Leader,PI,Energy,Preferred dates,Unacceptable,Comment
Application,,,,,,,,,,,,,,,
11016412,I-20221356,Nanotomography (EH1),2,12,12,12,,,Moisture dependent 3D shape respond of monocot fiber cell walls,Hesse,Hesse,17kev,13.01.2023,,
11016342,I-20221293 EC,Nanotomography (EH1),7,9,9,9,,,Peripheral nervous system features in chemotherapy-induced neuropathy models,Zippo,Zippo,25 Kev,,,
11015833,I-20220815,Nanotomography (EH1),12,15,9,9,,,Leaching by design: synergy between smart nanocontainers and inorganic extenders for active corrosion protection of AA2024,Konchakova,Konchakova,11keV,06.03.2022,02.02-05.03; 27.03-16.04; 25.04-07.05; 27.05-04.06; 10-13.06; 20-26.06; 01-31.07,
11016480,I-20221419,Nanotomography (EH1),14,18,12,12,,,Three-dimensional characterization of specific soft-tissue X-ray staining protocols by near-field ptychography,Greving,Herzen,17,,,
11016186,I-20221158,Nanotomography (EH1),17,9,9,9,,,Correlative high resolution imaging of Mg-Y-Zn based alloys,Tolnai,Tolnai,25-40 keV,,"15.02-15.03, 15.04-15.05",
11015933,I-20220924,Nanotomography (EH1),20,15,12,12,,,3D X-ray messages from the past to inform about future marine environments,Ni,Ni,17ke,13.02-08.05.2023,07-11.02; 15-22.05; 26-30.06,
11016352,I-20221295,Nanotomography (EH1),24,3,3,3,,,On springtails (Collembola): phylogenetic history and the diversification of the abdominal appendages,Oliveira,Oliveira,33KeV,"January, from 15th March or April",01.02-15.03,
11016103,I-20221075,Nanotomography (EH1),25,9,9,9,,,Homogeneous flow and deformation of metallic glasses under high pressure,Zeng,Xu,8 keV,,,
11016148,I-20221121 EC,Nanotomography (EH1),27,9,9,9,,,Tomography of nanoscale gradients in piezo-resistive cellulose textiles,Soederberg,Gordeyeva,11 keV,February,,


In [441]:
# preassign nano and micro
sch.init()
#sch.inject_data(dt.date(2023,2,22), dt.date(2023,3,10), "Setup", "micro")
#sch.inject_data(dt.date(2023,4,13), dt.date(2023,5,21), "Setup", "micro")
#sch.inject_data(dt.date(2023,6,22), dt.date(2023,7,16), "Setup", "micro")

sch.inject_data(dt.date(2023,3,16), dt.date(2023,4,2), "Setup", "nano")
sch.inject_data(dt.date(2023,6,1), dt.date(2023,6,20), "Setup", "nano")

sch.eval_proposals(pro_run1, clear=False)
sch.show(startdate=dt.datetime(2023, 2, 10), enddate=dt.datetime(2023, 8, 1))

,School holidays,P3 modes,P3 info,Setup,Proposal,Application,Leader,PI,Local contact,Absence,Extra hours,Title,Conferences,Comment
2023-02-10,,,IB,,,,,,,,,,,
2023-02-11,,,IB (tr),,,,,,,,,,,
2023-02-12,,,IB (tr),,,,,,,,,,,
2023-02-13,,,IB,,,,,,,,,,,
2023-02-14,,,IB,,,,,,,,,,,
2023-02-15,,A,IB tr,,,,,,,,,,,
2023-02-16,,,IB tr,,,,,,,,,,,
2023-02-17,,,IB tr,,,,,,,,,,,
2023-02-18,,tr,,,,,,,,,,,,
2023-02-19,,tr,,,,,,,,,,,,


# Write tables to excel file

In [202]:
if 'Unacceptable list' in pro.table:
    pro.table.drop("Unacceptable list", axis=1, inplace=True)
with pd.ExcelWriter("/home/fwilde/tmp/p05_schedule_{}.xlsx".format(year), date_format='YYYY-MM-DD', datetime_format='YYYY-MM-DD') as writer:
    sch.schedule.to_excel(writer, sheet_name="schedule", index=True, freeze_panes=(1,3))
    pro.table.to_excel(writer, sheet_name="proposals_1", index=True, freeze_panes=(1,1))

# Test zone

In [520]:
r = pd.date_range(dt.datetime(2023,1,1), dt.datetime(2023,2,1))
len(r)

32

In [391]:
nano = pro_run1.table[pro_run1.table['Setup'].str.contains('Nanotomography')]
nano = nano[(nano["Proposal"].str.contains("nano comm") == False)]
nano.sort_values(by=["Ranking (numbers)"]).loc[:, "Shifts final"].cumsum()

Application
11016412    12
11016342    21
11015833    30
11016480    42
11016186    51
11015933    63
11016352    66
11016103    75
11016148    84
Name: Shifts final, dtype: object

In [412]:
micro = pro_run1.table[pro_run1.table['Setup'].str.contains('Microtomography')]
micro = micro[(micro["Proposal"].str.contains("micro comm") == False)]
micro.sort_values(by=["Ranking (numbers)"]).loc[:, "Shifts final"].cumsum()
11016383 in micro.index.to_series()

False